# Gruppo 8A
Ennio Malvati - 484998 <br>
Gianluca Di Lorenzo - 583630 <br>
Raffaele Scarano - 576304

# Parallelizzazione della funzione ***frag_faces***

### Attiviamo l'ambiente con i packages necessari

In [1]:
using Pkg
Pkg.activate("./../../TopologicalGiftWrapping8A")

  Activating project at `~/.julia/environments/v1.7`


#### Carichiamo le librerie utili all'implementazione ed alla fase di test

In [2]:
using Profile
using ProfileView
using BenchmarkTools
using SparseArrays
using LinearAlgebra
using LinearAlgebraicRepresentation
using NearestNeighbors
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

### Settiamo Julia per runnare con più Threads

In [3]:
Threads.nthreads() = 8

In [4]:
Threads.nthreads()

8

#### Creaimo una funzione che ritorna sempre gli stessi valori che saranno poi utilizzati come input

In [5]:
function get_input()
    V = [0.6540618 0.2054992 0.2972308; 0.7142365 0.1455625 0.969203; 0.5941251 0.8769965 0.3624924; 0.6542998 0.8170598 1.0344647; 1.3260341 0.2707609 0.2428771; 1.3862088 0.2108241 0.9148494; 1.2660973 0.9422582 0.3081388; 1.326272 0.8823214 0.980111; -0.3874063 0.4902226 0.4536339; 0.3249123 0.707347 0.5231232; -0.1702819 -0.0864242 0.0297177; 0.5420367 0.1307001 0.099207; -0.317917 0.0663064 1.0658723; 0.3944016 0.2834308 1.1353616; -0.1007926 -0.5103404 0.6419561; 0.611526 -0.2932161 0.7114454; 0.7899026 0.0605793 0.6679889; 0.46601 0.0749997 0.6686316; 0.804323 0.3844725 0.6679746; 0.4804304 0.3988929 0.6686173; 0.7905452 0.060565 0.9922023; 0.4666527 0.0749854 0.992845; 0.8049656 0.3844582 0.992188; 0.4810731 0.3988786 0.9928307; -0.2261907 -0.0720455 0.4715635; -0.0499888 0.0863489 0.7965885; -0.0677963 0.219164 0.24378; 0.1084056 0.3775584 0.5688049; 0.0988343 -0.2998291 0.4063673; 0.2750362 -0.1414347 0.7313923; 0.2572286 -0.0086196 0.1785838; 0.4334306 0.1497748 0.5036087];

    EV = SparseArrays.sparse([1, 5, 9, 1, 6, 10, 2, 5, 11, 2, 6, 12, 3, 7, 9, 3, 8, 10, 4, 7, 11, 4, 8, 12, 13, 17, 21, 13, 18, 22, 14, 17, 23, 14, 18, 24, 15, 19, 21, 15, 20, 22, 16, 19, 23, 16, 20, 24, 25, 29, 33, 25, 30, 34, 26, 29, 35, 26, 30, 36, 27, 31, 33, 27, 32, 34, 28, 31, 35, 28, 32, 36], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1]);

    FE = SparseArrays.sparse([1, 3, 1, 4, 2, 3, 2, 4, 1, 5, 1, 6, 2, 5, 2, 6, 3, 5, 3, 6, 4, 5, 4, 6, 7, 9, 7, 10, 8, 9, 8, 10, 7, 11, 7, 12, 8, 11, 8, 12, 9, 11, 9, 12, 10, 11, 10, 12, 13, 15, 13, 16, 14, 15, 14, 16, 13, 17, 13, 18, 14, 17, 14, 18, 15, 17, 15, 18, 16, 17, 16, 18], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36], Int8[1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1]);
    return V, EV, FE
end

get_input (generic function with 1 method)

### Eseguiamo un benchmark della funziona frag face di Lar

Testiamo la funzione con l'utilizzo di @btime della libreria BenchmarkTools e successivamente con @benchmark

In [7]:
V, EV, FE = get_input()
@btime Lar.Arrangement.merge_vertices($Lar.Points(V),$Lar.ChainOp(EV),$Lar.ChainOp(FE),$1e-8)

  95.500 μs (2979 allocations: 200.70 KiB)


([0.6540618 0.2054992 0.2972308; 0.7142365 0.1455625 0.969203; … ; 0.2572286 -0.0086196 0.1785838; 0.4334306 0.1497748 0.5036087], sparse([1, 5, 9, 1, 6, 10, 2, 5, 11, 2  …  33, 27, 32, 34, 28, 31, 35, 28, 32, 36], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  21, 22, 22, 22, 23, 23, 23, 24, 24, 24], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1  …  1, 1, -1, 1, -1, 1, 1, 1, 1, 1], 36, 32), sparse([1, 3, 1, 4, 2, 3, 2, 4, 1, 5  …  14, 18, 15, 17, 15, 18, 16, 17, 16, 18], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  32, 32, 33, 33, 34, 34, 35, 35, 36, 36], Int8[1, 1, -1, 1, 1, -1, -1, -1, -1, 1  …  1, -1, -1, -1, 1, -1, -1, 1, 1, 1], 18, 36))

In [9]:
V, EV, FE = get_input()
@benchmark Lar.Arrangement.merge_vertices($Lar.Points(V),$Lar.ChainOp(EV),$Lar.ChainOp(FE),$1e-8)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   95.291 μs …   3.111 ms  ┊ GC (min … max):  0.00% … 95.72%
 Time  (median):      98.167 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   112.574 μs ± 194.207 μs  ┊ GC (mean ± σ):  11.52% ±  6.43%

     ▅█▄▂                                                        
  ▂▄██████▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂ ▃
  95.3 μs          Histogram: frequency by time          121 μs <

 Memory estimate: 200.70 KiB, allocs estimate: 2979.

### Proviamo una parallelizzazione utilizzando la macro @Threads

In [10]:
function merge_vertices_multithread(V::Lar.Points, EV::Lar.ChainOp, FE::Lar.ChainOp, err=1e-4)
    vertsnum = size(V, 1)
    edgenum = size(EV, 1)
    facenum = size(FE, 1)
    newverts = zeros(Int, vertsnum)

    # KDTree constructor needs an explicit array of Float64
    V = Array{Float64,2}(V)
    W = convert(Lar.Points, LinearAlgebra.transpose(V))
    kdtree = KDTree(W)

    # remove vertices congruent to a single representative
    todelete = []
    i = 1

    for vi in 1:vertsnum #non è possibile parallelizzare
        if !(vi in todelete)
            nearvs =  Lar.inrange(kdtree, V[vi, :], err)
            newverts[nearvs] .= i
            nearvs =  setdiff(nearvs, vi)
            todelete = union(todelete, nearvs)
            i = i + 1
        end
    end
    nV = V[setdiff(collect(1:vertsnum), todelete), :]
    
    # translate edges to take congruence into account
    edges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    oedges = Array{Tuple{Int, Int}, 1}(undef, edgenum)

    for ei in 1:edgenum #non è possibile parallelizzare
        v1, v2 = EV[ei, :].nzind
        edges[ei] = Tuple{Int, Int}(sort([newverts[v1], newverts[v2]]))
        oedges[ei] = Tuple{Int, Int}(sort([v1, v2]))
    end

    nedges = union(edges)
    # remove edges of zero length
    nedges = filter(t->t[1]!=t[2], nedges)
    nedgenum = length(nedges)
    nEV = spzeros(Int8, nedgenum, size(nV, 1))
 
    etuple2idx = Dict{Tuple{Int, Int}, Int}()

    for ei in 1:nedgenum #non è possibile parallelizzare
        begin
            nEV[ei, collect(nedges[ei])] .= 1
            nEV
        end
        etuple2idx[nedges[ei]] = ei
    end
    
    Threads.@threads for e in 1:nedgenum 
        v1,v2 = findnz(nEV[e,:])[1]
        nEV[e,v1] = -1; nEV[e,v2] = 1
    end
 
    # compute new faces to take congruence into account
    faces = [[
        map(x->newverts[x], FE[fi, ei] > 0 ? oedges[ei] : reverse(oedges[ei]))
        for ei in FE[fi, :].nzind
    ] for fi in 1:facenum]
 
 
    visited = []
    function filter_fn(face)
 
        verts = []
        map(e->verts = union(verts, collect(e)), face)
        verts = Set(verts)
 
        if !(verts in visited)
            push!(visited, verts)
            return true
        end
        return false
    end
 
    nfaces = filter(filter_fn, faces)
 
    nfacenum = length(nfaces)
    nFE = spzeros(Int8, nfacenum, size(nEV, 1))
 
    Threads.@threads for fi in 1:nfacenum
        Threads.@threads for edge in nfaces[fi]
            ei = etuple2idx[Tuple{Int, Int}(sort(collect(edge)))]
            nFE[fi, ei] = sign(edge[2] - edge[1])
        end
    end
 
    return Lar.Points(nV), nEV, nFE
 end

merge_vertices_multithread (generic function with 2 methods)

In [22]:
V, EV, FE = get_input()
@btime merge_vertices_multithread($Lar.Points(V),$Lar.ChainOp(EV),$Lar.ChainOp(FE),$1e-8)

  96.875 μs (2999 allocations: 202.24 KiB)


([0.6540618 0.2054992 0.2972308; 0.7142365 0.1455625 0.969203; … ; 0.2572286 -0.0086196 0.1785838; 0.4334306 0.1497748 0.5036087], sparse([1, 5, 9, 1, 6, 10, 2, 5, 11, 2  …  33, 27, 32, 34, 28, 31, 35, 28, 32, 36], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  21, 22, 22, 22, 23, 23, 23, 24, 24, 24], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1  …  1, 1, -1, 1, -1, 1, 1, 1, 1, 1], 36, 32), sparse([1, 3, 1, 4, 2, 3, 2, 4, 1, 5  …  14, 18, 15, 17, 15, 18, 16, 17, 16, 18], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  32, 32, 33, 33, 34, 34, 35, 35, 36, 36], Int8[1, 1, -1, 1, 1, -1, -1, -1, -1, 1  …  1, -1, -1, -1, 1, -1, -1, 1, 1, 1], 18, 36))

In [24]:
V, EV, FE = get_input()
@benchmark merge_vertices_multithread($Lar.Points(V),$Lar.ChainOp(EV),$Lar.ChainOp(FE),$1e-8)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   97.083 μs …   4.149 ms  ┊ GC (min … max):  0.00% … 96.28%
 Time  (median):     100.334 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   118.666 μs ± 245.354 μs  ┊ GC (mean ± σ):  13.79% ±  6.46%

    █▆▃                                                          
  ▂▇████▇▆▆▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  97.1 μs          Histogram: frequency by time          136 μs <

 Memory estimate: 202.24 KiB, allocs estimate: 2999.

Non notiamo nessuna differenza tra le due funzioni, proviamo utilizzando la macro @async

In [25]:
function merge_vertices_async(V::Lar.Points, EV::Lar.ChainOp, FE::Lar.ChainOp, err=1e-4)
    vertsnum = size(V, 1)
    edgenum = size(EV, 1)
    facenum = size(FE, 1)
    newverts = zeros(Int, vertsnum)

    # KDTree constructor needs an explicit array of Float64
    V = Array{Float64,2}(V)
    W = convert(Lar.Points, LinearAlgebra.transpose(V))
    kdtree = KDTree(W)

    # remove vertices congruent to a single representative
    todelete = []
    i = 1

    for vi in 1:vertsnum #non è possibile utilizzare @async
        if !(vi in todelete)
            nearvs =  Lar.inrange(kdtree, V[vi, :], err)
            newverts[nearvs] .= i
            nearvs =  setdiff(nearvs, vi)
            todelete = union(todelete, nearvs)
            i = i + 1
        end
    end
    nV = V[setdiff(collect(1:vertsnum), todelete), :]
    
    # translate edges to take congruence into account
    edges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    oedges = Array{Tuple{Int, Int}, 1}(undef, edgenum)

    for ei in 1:edgenum #non è possibile utilizzare @async
        v1, v2 = EV[ei, :].nzind
        edges[ei] = Tuple{Int, Int}(sort([newverts[v1], newverts[v2]]))
        oedges[ei] = Tuple{Int, Int}(sort([v1, v2]))
    end

    nedges = union(edges)
    # remove edges of zero length
    nedges = filter(t->t[1]!=t[2], nedges)
    nedgenum = length(nedges)
    nEV = spzeros(Int8, nedgenum, size(nV, 1))
 
    etuple2idx = Dict{Tuple{Int, Int}, Int}()

    for ei in 1:nedgenum #non è possibile utilizzare @async
        begin
            nEV[ei, collect(nedges[ei])] .= 1
            nEV
        end
        etuple2idx[nedges[ei]] = ei
    end
    
    @async for e in 1:nedgenum 
        v1,v2 = findnz(nEV[e,:])[1]
        nEV[e,v1] = -1; nEV[e,v2] = 1
    end
 
    # compute new faces to take congruence into account
    faces = [[
        map(x->newverts[x], FE[fi, ei] > 0 ? oedges[ei] : reverse(oedges[ei]))
        for ei in FE[fi, :].nzind
    ] for fi in 1:facenum]
 
 
    visited = []
    function filter_fn(face)
 
        verts = []
        map(e->verts = union(verts, collect(e)), face)
        verts = Set(verts)
 
        if !(verts in visited)
            push!(visited, verts)
            return true
        end
        return false
    end
 
    nfaces = filter(filter_fn, faces)
 
    nfacenum = length(nfaces)
    nFE = spzeros(Int8, nfacenum, size(nEV, 1))
 
    @async for fi in 1:nfacenum
        @async for edge in nfaces[fi]
            ei = etuple2idx[Tuple{Int, Int}(sort(collect(edge)))]
            nFE[fi, ei] = sign(edge[2] - edge[1])
        end
    end
 
    return Lar.Points(nV), nEV, nFE
 end

merge_vertices_async (generic function with 2 methods)

In [27]:
V, EV, FE = get_input()
@btime merge_vertices_async(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),1e-8)

  80.750 μs (2622 allocations: 174.80 KiB)


([0.6540618 0.2054992 0.2972308; 0.7142365 0.1455625 0.969203; … ; 0.2572286 -0.0086196 0.1785838; 0.4334306 0.1497748 0.5036087], sparse([1, 5, 9, 1, 6, 10, 2, 5, 11, 2  …  33, 27, 32, 34, 28, 31, 35, 28, 32, 36], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  21, 22, 22, 22, 23, 23, 23, 24, 24, 24], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1  …  1, 1, -1, 1, -1, 1, 1, 1, 1, 1], 36, 32), sparse([1, 3, 1, 4, 2, 3, 2, 4, 1, 5  …  14, 18, 15, 17, 15, 18, 16, 17, 16, 18], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  32, 32, 33, 33, 34, 34, 35, 35, 36, 36], Int8[1, 1, -1, 1, 1, -1, -1, -1, -1, 1  …  1, -1, -1, -1, 1, -1, -1, 1, 1, 1], 18, 36))

In [29]:
V, EV, FE = get_input()
@benchmark merge_vertices_async(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),1e-8)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   80.542 μs …  36.894 ms  ┊ GC (min … max):  0.00% … 99.59%
 Time  (median):      84.958 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   109.391 μs ± 847.495 μs  ┊ GC (mean ± σ):  20.36% ±  2.63%

    ▂▆█▆▃▂                                                       
  ▂▅████████▇▆▆▅▄▄▄▃▃▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  80.5 μs          Histogram: frequency by time          119 μs <

 Memory estimate: 174.80 KiB, allocs estimate: 2622.

In questo caso, come per frag_face i tempi sono ridotti quasi del 25% e l'allocazione di memoria è ridotta quasi del 20%.
Considerando che anche questo metodo viene utilizzato più volte nel macrotask spatial_arrangement, possiamo ritenere questo risultato soddisfacente.

In [30]:
Profile.clear()
Profile.init(delay=.1e-7)
@profile for _ in 1:200
    merge_vertices_async(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),1e-8)
end

In [31]:
ProfileView.view()
Profile.print()

Overhead ╎ [+additional indent] Count File:Line; Function
  2╎2    In[25]:69; (::var"#37#46")(e::Tuple{Int64,...
   ╎1    In[25]:33; merge_vertices_async(V::Matrix{...
   ╎ 1    @Base/sort.jl:770; sort
   ╎  1    @Base/sort.jl:770; #sort#9
   ╎   1    ...e/abstractarray.jl:1129; copymutable
   ╎    1    @Base/array.jl:373; similar
  1╎     1    @Base/boot.jl:457; Array
   ╎2    @Base/abstractarray.jl:1433; mightalias(A::SparseMatrixCSC{I...
   ╎ 1    @Base/abstractarray.jl:1443; _isdisjoint
  1╎  1    @Base/tuple.jl:0; in
   ╎ 1    @Base/reflection.jl:553; isbits
  1╎  1    @Base/Base.jl:37; getproperty
  4╎4    @Base/abstractset.jl:279; issubset(l::Set{Any}, r::Set{Any})
  1╎1    @Base/abstractset.jl:293; issubset(l::Set{Any}, r::Set{Any})
  2╎2    @Base/abstractset.jl:442; mapfilter(pred::Base.var"#115#1...
  2╎2    @Base/abstractset.jl:50; union(s::Vector{Any}, sets::Vec...
  1╎1    @Base/abstractset.jl:97; union!(s::Set{Any}, itr::Vector...
  3╎3    @Base/array.jl:325; _copyto_impl

   ╎    ╎    ╎  22   @Base/array.jl:2632; _shrink
   ╎    ╎    ╎   22   @Base/set.jl:23; Set
   ╎    ╎    ╎    22   @Base/set.jl:25; _Set
   ╎    ╎    ╎     22   @Base/set.jl:10; Set
   ╎    ╎    ╎    ╎ 1    ...bstractset.jl:95; union!(s::Set{Int64}, ...
   ╎    ╎    ╎    ╎  1    @Base/set.jl:57; length
  1╎    ╎    ╎    ╎   1    @Base/Base.jl:42; getproperty
   ╎    ╎    ╎    ╎ 3    ...bstractset.jl:97; union!(s::Set{Int64}, ...
   ╎    ╎    ╎    ╎  3    @Base/set.jl:59; push!
   ╎    ╎    ╎    ╎   1    @Base/dict.jl:382; setindex!(h::Dict{Int...
   ╎    ╎    ╎    ╎    1    @Base/dict.jl:314; ht_keyindex2!(h::Dic...
   ╎    ╎    ╎    ╎     1    @Base/dict.jl:171; isslotempty
  1╎    ╎    ╎    ╎    ╎ 1    @Base/array.jl:861; getindex
  1╎    ╎    ╎    ╎   1    @Base/dict.jl:384; setindex!(h::Dict{Int...
  1╎    ╎    ╎    ╎   1    @Base/dict.jl:392; setindex!(h::Dict{Int...
   ╎    ╎    ╎    ╎ 1    ...bstractset.jl:98; union!(s::Set{Int64}, ...
   ╎    ╎    ╎    ╎  1    @Base/set.jl:57;

  9╎    ╎    ╎     9    @Base/boot.jl:457; Array
   ╎    ╎     2    In[25]:37; merge_vertices_async(V::M...
   ╎    ╎    ╎ 2    ...e/abstractset.jl:50; union
   ╎    ╎    ╎  2    @Base/array.jl:2612; union!(v::Vector{Tuple{I...
   ╎    ╎    ╎   2    @Base/array.jl:2607; _grow!
   ╎    ╎    ╎    2    ...abstractset.jl:444; mapfilter(pred::Base.va...
   ╎    ╎    ╎     1    @Base/array.jl:835; iterate
  1╎    ╎    ╎    ╎ 1    @Base/array.jl:215; length
   ╎    ╎    ╎     1    @Base/array.jl:995; push!
  1╎    ╎    ╎    ╎ 1    @Base/array.jl:948; _growend!
   ╎    ╎     1    In[25]:41; merge_vertices_async(V::M...
   ╎    ╎    ╎ 1    ...sparsematrix.jl:1700; spzeros
   ╎    ╎    ╎  1    ...sparsematrix.jl:1703; spzeros(#unused#::Type{I...
   ╎    ╎    ╎   1    ...sparsematrix.jl:38; SparseMatrixCSC(m::Int64...
   ╎    ╎    ╎    1    ...sparsematrix.jl:66; sparse_check
   ╎    ╎    ╎     1    @Base/range.jl:837; iterate
  1╎    ╎    ╎    ╎ 1    .../promotion.jl:468; ==
   ╎    ╎     1    I

   ╎    ╎    ╎    ╎   4    @Base/sort.jl:184; searchsortedfirst
  4╎    ╎    ╎    ╎    4    @Base/array.jl:861; getindex
  2╎    ╎    ╎    ╎  3    ...arsevector.jl:598; getindex(A::SparseMat...
  1╎    ╎    ╎    ╎   1    @Base/array.jl:861; getindex
   ╎    ╎    ╎    ╎  3    ...arsevector.jl:599; getindex(A::SparseMat...
   ╎    ╎    ╎    ╎   3    @Base/array.jl:995; push!
  3╎    ╎    ╎    ╎    3    @Base/array.jl:948; _growend!
   ╎    ╎    ╎    ╎  1    ...arsevector.jl:600; getindex(A::SparseMat...
   ╎    ╎    ╎    ╎   1    @Base/array.jl:995; push!
  1╎    ╎    ╎    ╎    1    @Base/array.jl:948; _growend!
   ╎    ╎    ╎    ╎  3    ...arsevector.jl:603; getindex(A::SparseMat...
  3╎    ╎    ╎    ╎   3    @Base/range.jl:837; iterate
   ╎    ╎     357  In[25]:79; merge_vertices_async(V::M...
  1╎    ╎    ╎ 357  @Base/array.jl:2484; filter(f::var"#filter_fn...
   ╎    ╎    ╎  4    In[25]:68; (::var"#filter_fn#45"{Vec...
   ╎    ╎    ╎   4    @Base/array.jl:125; vect
   ╎    ╎    ╎    

   ╎    ╎    ╎    ╎     10   @Base/array.jl:649; collect
   ╎    ╎    ╎    ╎    ╎ 10   @Base/array.jl:655; _collect
   ╎    ╎    ╎    ╎    ╎  10   @Base/array.jl:605; _similar_for
   ╎    ╎    ╎    ╎    ╎   10   ...actarray.jl:785; similar
   ╎    ╎    ╎    ╎    ╎    10   ...ctarray.jl:794; similar
   ╎    ╎    ╎    ╎    ╎     10   @Base/boot.jl:466; Array
 10╎    ╎    ╎    ╎    ╎    ╎ 10   ...e/boot.jl:457; Array
 16╎    ╎    ╎  41   In[25]:70; (::var"#filter_fn#45"{Vec...
   ╎    ╎    ╎   25   @Base/set.jl:23; Set(itr::Vector{Any})
   ╎    ╎    ╎    25   @Base/set.jl:25; _Set
  1╎    ╎    ╎     25   @Base/set.jl:10; Set
   ╎    ╎    ╎    ╎ 2    ...abstractset.jl:95; union!(s::Set{Any}, it...
   ╎    ╎    ╎    ╎  2    @Base/set.jl:57; length
  2╎    ╎    ╎    ╎   2    @Base/Base.jl:42; getproperty
  5╎    ╎    ╎    ╎ 10   ...abstractset.jl:97; union!(s::Set{Any}, it...
   ╎    ╎    ╎    ╎  5    @Base/set.jl:59; push!(s::Set{Any}, x::...
   ╎    ╎    ╎    ╎   3    @Base/dict.jl:382; se